# Exploratory Data Analysis, Hop Teaming Dataset

### Import libraries, connect to the database, and take a look around

In [1]:
# import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [15]:
# Set the display to show more rows and columns
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 200

In [3]:
# Assign sqlite database to db variable
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [ ]:
# Check the tables in the database
tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for table in tables:
    print(table[0])

In [ ]:
# For reference: pull out the column names for a given table
col = db.execute("PRAGMA table_info(specialty)").fetchall()
print([e[1] for e in col])

### How many providers are in the nppes dataset?
*Note that `entity_type_code` 1 = Providers and `entity_type_code` 2 = Facilities.*

In [ ]:
all_providers = """
SELECT COUNT(npi), COUNT(DISTINCT npi)
FROM nppes
"""
all_providers = pd.read_sql(all_providers, db)

In [ ]:
all_providers

In [ ]:
# Create a query to get all providers with a practice location within the Nashville CBSA
nash_providers = """
    SELECT n.*

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
"""

# Read the query results into a dataframe
nash_nppes = pd.read_sql(nash_providers, db)

In [ ]:
# Take a look at the number of datapoints available
nash_nppes.info()
# There are 38,414 rows in the table

In [ ]:
# Take a look at the unique provider credential types
nash_nppes.provider_credential.unique().tolist()

*The credentials are VERY not-standardized and include multiple variants of the same thing with different spellings (e.g. `PharmD` vs. `Pharm.D.` vs. `PHARMD`) in addition to commas within the field itself (e.g. `APN-BC, NP-C`). If we need to use this field, we will need to do some data cleaning.*

In [ ]:
# Subset to only individual providers
ind_nash_nppes = nash_nppes[nash_nppes['entity_type_code'] == 1]

# Take a look at how many individual providers there are in the Nashville area
ind_nash_nppes.info()

In [ ]:
# Who is the provider without a last name?
ind_nash_nppes[ind_nash_nppes['provider_last_name'].isna() == True]

*Hilariously, [this is a real provider](https://npiregistry.cms.hhs.gov/registry/provider-view/1215064852) whose last name is "Null".*

### Explore the specialties data

In [ ]:
specialties_raw = """
    SELECT *
    FROM specialty
"""
specialties_raw = pd.read_sql(specialties_raw, db)

In [ ]:
specialties_raw

In [ ]:
specialties = """
    SELECT n.*
        , s.primary_taxonomy
        , s.Classification

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
    
    --WHERE entity_type_code = 1 -- only get individuals
"""
# Read the query results into a dataframe
nash_nppes = pd.read_sql(specialties, db)

In [ ]:
nash_nppes.info()

In [ ]:
nash_nppes.to_csv('../data/nashville_providers_with_specialty.csv', index = False)

In [ ]:
# Take a look at the individual providers who do not have a primary specialty
nash_nppes[(nash_nppes['primary_taxonomy'].isna() == True)
           & (nash_nppes['entity_type_code'] == 1)]#.nunique() # Add this to count them

*There are 104 individual providers who do not have a specialty. At a quick glance, there is no clear connection between these providers - they are in different areas, have different degrees, etc.*

In [ ]:
# Take a look at the types of organizations in the dataset
# There are 152 unique classifications plus null values
nash_nppes[nash_nppes['entity_type_code'] == 2].groupby('Classification')['npi'].count().sort_values(ascending = False)

In [ ]:
# Count the number of organizations that do not have a primary specialty
nash_nppes[(nash_nppes['primary_taxonomy'].isna() == True)
           & (nash_nppes['entity_type_code'] == 2)].nunique() # Add this to count them
# There are 85 organizations that do not have a specialty

### Follow just one NPI through the tables

In [ ]:
sample = """
WITH nashville_providers AS (
    SELECT n.*
        , s.Classification AS specialty

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
    
    WHERE n.npi = 1215064852 -- Limit to Elizabeth Null only
)
,

referrals_to AS (
    SELECT n.npi
    , n.specialty
    , rto.*
    
    FROM nashville_providers AS n

    JOIN nashville_referrals AS rto
        ON rto.to_npi = n.npi
)
,

referrals_from AS (
    SELECT n.npi
    , n.specialty
    , rfrom.*

    FROM nashville_providers AS n

    JOIN nashville_referrals AS rfrom
        ON rfrom.from_npi = n.npi
)
SELECT *

FROM referrals_to

UNION

SELECT *
FROM referrals_from
;
"""

# Read the query results into a dataframe
liz_null = pd.read_sql(sample, db)

In [ ]:
# Take a look at the sample provider dataframe
liz_null

### Take a look at the total hop-teaming data joined up

In [ ]:
nashville_referrals_agg = """
WITH nashville_providers AS (
    SELECT n.*
        , s.Classification AS specialty

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
)
,

referrals_to AS (
    SELECT rto.*
    , s.Classification AS from_npi_specialty
    , n.specialty AS to_npi_specialty
    
    FROM nashville_providers AS n

    JOIN nashville_referrals AS rto
        ON rto.to_npi = n.npi
    
    LEFT JOIN specialty AS s
        ON s.npi = rto.from_npi
    
    WHERE n.entity_type_code = 2 -- only get referrals made *to* organizations
)
,

referrals_from AS (
    SELECT rfrom.*
    , n.specialty AS from_npi_specialty
    , s.Classification AS to_npi_specialty

    FROM nashville_providers AS n

    JOIN nashville_referrals AS rfrom
        ON rfrom.from_npi = n.npi
    
    LEFT JOIN specialty AS s
        ON s.npi = rfrom.to_npi

    WHERE n.entity_type_code = 1 -- only get referrals made *by* individual providers
)
,

tot AS (
SELECT *

FROM referrals_to

UNION

SELECT *
FROM referrals_from
)

SELECT COUNT(*) AS tot_rows
    , COUNT(DISTINCT t.from_npi) AS unique_from_npis
    , COUNT(DISTINCT t.to_npi) AS unique_to_npis
    
FROM tot AS t
;
"""
nashville_referrals_agg = pd.read_sql(nashville_referrals_agg, db)

In [ ]:
nashville_referrals_agg

*N.B.: The unique number of provider organizations (that is, entity type code 2) is 2.5 times the number of entities within the Nashville CBSA. This would be the case in Nashville area providers are referring to healthcare organizations outside of the Nashville CBSA...*

In [ ]:
# Write a query to get every unique from-to NPI pair where
# The from_npi is an individual provider
# The to_npi is an organization
# And one or both of the NPIs are located in the Nashville CBSA

nashville_referrals = """
WITH nashville_providers AS (
    SELECT n.*
        , s.Classification AS specialty

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
)
,

referrals_to AS (
    SELECT rto.*
    , s.Classification AS from_npi_specialty
    , n.specialty AS to_npi_specialty
    
    FROM nashville_providers AS n

    JOIN nashville_referrals AS rto
        ON rto.to_npi = n.npi
    
    LEFT JOIN specialty AS s
        ON s.npi = rto.from_npi
    
    WHERE n.entity_type_code = 2 -- only get referrals made *to* organizations
)
,

referrals_from AS (
    SELECT rfrom.*
    , n.specialty AS from_npi_specialty
    , s.Classification AS to_npi_specialty

    FROM nashville_providers AS n

    JOIN nashville_referrals AS rfrom
        ON rfrom.from_npi = n.npi
    
    LEFT JOIN specialty AS s
        ON s.npi = rfrom.to_npi

    WHERE n.entity_type_code = 1 -- only get referrals made *by* individual providers
)

SELECT *

FROM referrals_to

UNION -- use this to ensure that the results are de-duplicated

SELECT *

FROM referrals_from

;
"""
nashville_referrals = pd.read_sql(nashville_referrals, db)

In [ ]:
# Take a look at the shape of the resulting dataset
nashville_referrals.info()

In [ ]:
# Double check that the counts line up with the aggregate query above
nashville_referrals.from_npi.nunique()

In [ ]:
# Double check that the counts line up with the aggregate query above
nashville_referrals.to_npi.nunique()

In [ ]:
nashville_referrals.head(100)

*This dataset has all of the unique from-to NPI pairs where the from_npi is an individual provider, the to_npi is an organization, and one or both of them is located in the Nashville CBSA. The query shows that there are 174,382 unique from-to NPI pairs in Nashville. Once we validate the query, we should be able to use the resulting dataframe as a base for visualizing how different types of providers interact.*

In [ ]:
# Write out the complete dataset to a csv to upload into neo4j
nashville_referrals.to_csv('../data/nashville_referrals.csv', index = False)

### Limit to only referrals where the `from_npi` and `to_npi` are both in the Nashville CBSA
*For import into neo4j.*

In [24]:
# Write a query to get every unique from-to NPI pair where
# The from_npi is an individual provider
# The to_npi is an organization
# Both of the NPIs are located in the Nashville CBSA

nashville_referrals_internal = """
WITH nashville_providers AS (
    SELECT n.*
        , s.Classification AS specialty

    FROM nppes AS n

    JOIN cbsa AS c
        ON c.ZIP = CAST(n.zip_5 AS INTEGER)
    
    LEFT JOIN specialty AS s
        ON s.npi = n.npi
        
    WHERE state IN ('TN', 'TENNESSEE')
)

SELECT r.*
, CAST(n2.zip_5 AS INTEGER) AS from_zip
, n2.specialty AS from_npi_specialty
, n2.entity_type_code AS from_entity_type_code
, CAST(n.zip_5 AS INTEGER) AS to_zip
, n.specialty AS to_npi_specialty
, n.entity_type_code AS to_entity_type_code

FROM nashville_referrals AS r

JOIN nashville_providers AS n
    ON n.npi = r.to_npi

JOIN nashville_providers AS n2
    ON n2.npi = r.from_npi

-- TONY: Comment out the filters below to prepare the data for neo4j
WHERE n.entity_type_code = 2 -- only get referrals made *to* organizations
AND n2.entity_type_code = 1 -- only get referrrals made *by* individuals
AND r.average_day_wait >= 2 -- filter out emergency referrals

"""
nashville_referrals_internal = pd.read_sql(nashville_referrals_internal, db)

In [25]:
nashville_referrals_internal

,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_zip,from_npi_specialty,from_entity_type_code,to_zip,to_npi_specialty,to_entity_type_code
0,20662650,1780832899,1245233220,62,63,2.667,20.412,37203,Anesthesiology,1.0,37115,Clinic/Center,2.0
1,16742214,1851362628,1790788040,26,65,3.231,18.277,37129,Internal Medicine,1.0,37129,"Hospice Care, Community Based",2.0
2,6532675,1396753356,1609879956,27,68,4.529,27.525,37174,Internal Medicine,1.0,37214,"Hospice Care, Community Based",2.0
3,18197644,1144264458,1609879956,36,77,3.247,27.359,37067,Family Medicine,1.0,37214,"Hospice Care, Community Based",2.0
4,26837443,1942347513,1609879956,36,53,21.509,40.591,37217,Nurse Practitioner,1.0,37214,"Hospice Care, Community Based",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34718,5795636,1376635052,1063947141,55,66,32.212,51.082,38474,Family Medicine,1.0,37067,Optometrist,2.0
34719,5245750,1497966105,1629598453,157,284,2.313,19.390,37209,Urology,1.0,37203,Urology,2.0
34720,26866273,1952388977,1629598453,242,408,2.368,21.669,37209,Urology,1.0,37203,Urology,2.0
34721,18227005,1174669998,1629593314,63,125,2.688,11.519,37087,Internal Medicine,1.0,37030,Skilled Nursing Facility,2.0


In [26]:
nashville_referrals_internal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34723 entries, 0 to 34722
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  34723 non-null  int64  
 1   from_npi               34723 non-null  int64  
 2   to_npi                 34723 non-null  int64  
 3   patient_count          34723 non-null  int64  
 4   transaction_count      34723 non-null  int64  
 5   average_day_wait       34723 non-null  float64
 6   std_day_wait           34723 non-null  float64
 7   from_zip               34723 non-null  int64  
 8   from_npi_specialty     34667 non-null  object 
 9   from_entity_type_code  34723 non-null  float64
 10  to_zip                 34723 non-null  int64  
 11  to_npi_specialty       34542 non-null  object 
 12  to_entity_type_code    34723 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 3.4+ MB


In [ ]:
nashville_referrals_internal['from_npi'].nunique()

In [ ]:
# Write out the complete dataset to a csv to upload into neo4j
nashville_referrals_internal.to_csv('../data/nashville_referrals_internal.csv', index = False)

In [ ]:
test = """
SELECT COUNT(DISTINCT zip_5)
FROM nppes
JOIN cbsa
    ON cbsa.ZIP = CAST(nppes.zip_5 AS INTEGER)
"""
test = pd.read_sql(test, db)

In [ ]:
test

In [19]:
zip_test = """
SELECT DISTINCT zip_5, zip_9

FROM nppes

WHERE state IN ('TN', 'TENNESSEE')

GROUP BY 1

HAVING (LENGTH(zip_9) = MIN(LENGTH(zip_9))
    OR LENGTH(zip_9) = MAX(LENGTH(zip_9))
    )
"""
zip_test = pd.read_sql(zip_test, db)

In [20]:
zip_test

,zip_5,zip_9
0,19107,191074405.0
1,27215,27215.0
2,30103,30103.0
3,31643,316434265.0
4,31827,318273234.0
5,32709,32709.0
6,32822,32822.0
7,34043,34043.0
8,34337,34337.0
9,34742,347422663.0
